# Sistema de Compressão

$ \frac{d\dot{m}}{dt} = \frac{A_1}{L_C}(\phi (N(t), \dot{m})P_1 - P_P(t)) $

$ \frac{d P_P}{dt} = \frac{C_1^2}{\nu _P}(\dot{m}(t) - \alpha (t) K_\nu \sqrt{P_P - P_{out}}) $

$ \begin{matrix} A_1 & = & 2.6\centerdot 10^{-3} m² \\
\nu _P & = & 2.0 m³ \\
L_C & = & 2.0 m \\
K_\nu & = & \frac{0.38 kg}{(kPa)^{0.5}s} \\
P_1 & = & 4.5 MPa \\
P{out} & = & 5.0 MPa \end{matrix}
$

$ \frac{d\dot{m}}{dt} = \frac{2.6\centerdot 10^{-3}}{2.0}(\phi (N(t), \dot{m})\centerdot 4.5 - P_P) $

$ \frac{d P_P}{dt} = \frac{479.029^2}{2.0}(\dot{m} - \alpha {0.38} \sqrt{P_P - 5.0}) $

#### Importações

In [ ]:
import numpy as np
from scipy.optimize import fsolve
import casadi as ca
import plotly.graph_objects as go
import optuna
from plotly.subplots import make_subplots
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from scipy.interpolate import griddata
from libs.Simulation import Simulation
from libs.Model import MyModel
from libs.Interpolation import Interpolation

#### Classes

##### Solução Numérica

##### Modelo de Rede Neural

#### Constantes e Variáveis Auxiliares

In [13]:
np.random.seed(42)
print(np.random.seed)

# Constantes
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38
P1 = 4.5
P_out = 5
C = 479

timestep = 3 # Passos no passado para prever o próximo
nAlphas = 20 # Número de vezes que o Alfa irá mudar, considere o treino e os testes.
alphas = np.random.uniform(0.35,0.65, nAlphas+1) # Abertura da válvula
N_RotS = np.random.uniform(27e3, 5e4, nAlphas+1)
epochs = 1000
nData = 600
nAlphasTeste = 7
nDataTeste = nData//nAlphasTeste
perturb = 1e-4
tempo = 60
tempoTeste = 60
dt = 0.1 # Tempo amostral

# Variáveis auxiliares
interval = [np.linspace(i * tempo, (i + 1) * tempo, nData) for i in range(nAlphas)]
interval_test = [np.linspace(i * tempoTeste, (i + 1) * tempoTeste, nDataTeste) for i in range(nAlphasTeste)]
massFlowrate = []
PlenumPressure = []
alpha_values = []
RNN_train = []
RNN_trainFut = []

<built-in function seed>


### Solução Numérica

##### Interpolador

##### Cálculo da Solução

In [19]:
lut = Interpolation('./tabela_phi.csv')
lut.load_data()
interpolation = lut.interpolate()
sim = Simulation(A1= A1, Lc= Lc, kv = kv,P1 = P1, P_out= P_out, C = C, alphas = alphas, N_RotS = N_RotS, nAlphas = nAlphas, nData = nData, perturb = perturb, tempo=tempo, dt = dt, interpolation = interpolation)
# Execute a simulação
sim.run()

RNN_train = sim.RNN_train
RNN_trainFut = sim.RNN_trainFut
massFlowrate = sim.massFlowrate
PlenumPressure = sim.PlenumPressure
alpha_values = sim.alpha_values
phi_values = sim.Phi_values

print(sim.time)

TypeError: Simulation.__init__() got an unexpected keyword argument 'N_RotS'

##### Gráfico do Modelo

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Vazão vs Tempo", "Pressão vs Tempo", "Alpha vs Tempo", "Phi vs Tempo"))

for i in range(0, nAlphas):
    # Vazão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines',
                             name='Vazão', legendgroup='massflow', showlegend=i == 0), row = 1, col = 1)
    # Pressão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines',
                             name='Pressão', legendgroup='pressure', showlegend=i == 0), row = 1, col = 2)
    # Alphas
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(alpha_values[i]), mode='lines', 
                             name='Alphas', line=dict(dash='dash'), legendgroup='alpha', showlegend=i == 0), row = 2, col = 1)
    # Phi
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(phi_values[i]), mode='lines', 
                             name='Alphas', line=dict(dash='dash'), legendgroup='alpha', showlegend=i == 0), row = 2, col = 2)

# Atualiza layout
fig.update_layout(
    xaxis_title='Tempo',
    grid=dict(rows=1, columns=3),
    template='plotly',
    showlegend=False,
    height = 600
)

# Mostra a figura
fig.show()


### Rede Neural

##### Dados de Treino

In [ ]:
RNN_train = np.array(RNN_train)


X_train = []
y_train = []

for i in range(len(RNN_train) - timestep):
    X_train.append(RNN_train[i:i + timestep])  
    if i + timestep < len(RNN_train):           
        y_train.append(RNN_train[i + timestep, :2])  

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

x_min = X_train.amin(dim=(0, 1), keepdim=True)
x_max = X_train.amax(dim=(0, 1), keepdim=True)
print(x_min)

y_train = y_train.unsqueeze(1)

tensor([[[3.8376e+00, 5.2552e+00, 3.5593e-01, 2.7907e+04]]])


/tmp/ipykernel_4261/2575451175.py:12: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)



In [ ]:
import optuna
from torch.utils.data import DataLoader

def objective(trial):
    # Definir os hiperparâmetros a serem otimizados
    units = trial.suggest_int("units", 10, 100, step=10)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    lr = 1e-5
    
    # Criar o modelo com o número de neurônios sugerido
    model = MyModel(units, A1, Lc, kv, P1, P_out, C, dt, x_min, x_max, interpolation)
    
    # Configurar o DataLoader com o batch_size sugerido
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Configurar o otimizador com a taxa de aprendizado sugerida
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Treinamento
    model.train()
    epochs = 100  # Escolha o número de épocas desejado para avaliação
    for epoch in range(epochs):
        total_loss = 0
        for inputs, y_true in train_loader:
            optimizer.zero_grad()
            y_pred = model(inputs)
            loss = model.loss_custom(y_true, y_pred, inputs)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    
    # Retornar o valor médio da perda como métrica de avaliação
    return total_loss / len(train_loader)

# Criar o estudo Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

# Resultados
print("Best trial:")
print(f"  Units: {study.best_params['units']}")
print(f"  Batch size: {study.best_params['batch_size']}")

[I 2024-12-03 22:35:07,641] A new study created in memory with name: no-name-c7c75da5-f451-4091-bd86-d9774137c593
[I 2024-12-03 22:39:49,600] Trial 0 finished with value: 1.171398748934269 and parameters: {'units': 20, 'batch_size': 16}. Best is trial 0 with value: 1.171398748934269.
[I 2024-12-03 22:43:04,420] Trial 1 finished with value: 1.2043261182308198 and parameters: {'units': 80, 'batch_size': 32}. Best is trial 0 with value: 1.171398748934269.
[I 2024-12-03 22:45:10,079] Trial 2 finished with value: 5.821079375896048 and parameters: {'units': 80, 'batch_size': 64}. Best is trial 0 with value: 1.171398748934269.
[I 2024-12-03 22:50:26,629] Trial 3 finished with value: 0.9488009311755499 and parameters: {'units': 60, 'batch_size': 16}. Best is trial 3 with value: 0.9488009311755499.
[I 2024-12-03 22:53:26,137] Trial 4 finished with value: 1.939399183511734 and parameters: {'units': 40, 'batch_size': 32}. Best is trial 3 with value: 0.9488009311755499.
[I 2024-12-03 22:55:27,642]

Best trial:
  Units: 60
  Batch size: 16


KeyError: 'lr'

##### Rede

##### Treinamento

In [ ]:
model = MyModel(60, A1, Lc, kv, P1, P_out, C, dt, x_min, x_max, interpolation)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size= 16, shuffle=True)
Adam = optim.Adam
model.train_model(model, train_loader, 1e-5 , 1000, Adam)

Epoch [1/1000], Loss: 806.6719322509765
Epoch [2/1000], Loss: 426.95941107177737
Epoch [3/1000], Loss: 141.24839690653482
Epoch [4/1000], Loss: 49.6592074025472
Epoch [5/1000], Loss: 29.124573286056517
Epoch [6/1000], Loss: 20.47976022275289
Epoch [7/1000], Loss: 16.193093282699586
Epoch [8/1000], Loss: 14.587980520248413
Epoch [9/1000], Loss: 13.766430392424265
Epoch [10/1000], Loss: 13.088649251143138
Epoch [11/1000], Loss: 12.496016995112102
Epoch [12/1000], Loss: 11.981640622456869
Epoch [13/1000], Loss: 11.534401554743448
Epoch [14/1000], Loss: 11.17324045642217
Epoch [15/1000], Loss: 10.84324596341451
Epoch [16/1000], Loss: 10.57062252521515
Epoch [17/1000], Loss: 10.32831611251831
Epoch [18/1000], Loss: 10.103574007670085
Epoch [19/1000], Loss: 9.905538476308188
Epoch [20/1000], Loss: 9.716125051021576
Epoch [21/1000], Loss: 9.545699194272359
Epoch [22/1000], Loss: 9.385939719518026
Epoch [23/1000], Loss: 9.240735281944275
Epoch [24/1000], Loss: 9.108221536795298
Epoch [25/1000]

In [ ]:
model.train_model(model, train_loader, 1e-5 , 3000, Adam)

Epoch [1/3000], Loss: 0.044065591392107306
Epoch [2/3000], Loss: 0.04405109031886483
Epoch [3/3000], Loss: 0.044142033429428316
Epoch [4/3000], Loss: 0.04430458689962203
Epoch [5/3000], Loss: 0.04403802603157237
Epoch [6/3000], Loss: 0.04370383668729725
Epoch [7/3000], Loss: 0.04411037410888821
Epoch [8/3000], Loss: 0.043959160702768715
Epoch [9/3000], Loss: 0.04405124828987755
Epoch [10/3000], Loss: 0.04399379632109776
Epoch [11/3000], Loss: 0.043796562471659854
Epoch [12/3000], Loss: 0.0436414464833991
Epoch [13/3000], Loss: 0.0435058715993849
Epoch [14/3000], Loss: 0.04373251343037312
Epoch [15/3000], Loss: 0.0439415720156394
Epoch [16/3000], Loss: 0.04376937397639267
Epoch [17/3000], Loss: 0.04368195112010775
Epoch [18/3000], Loss: 0.043808932482032104
Epoch [19/3000], Loss: 0.04372724757421141
Epoch [20/3000], Loss: 0.04344668875867501
Epoch [21/3000], Loss: 0.04347308815790651
Epoch [22/3000], Loss: 0.04358849378993424
Epoch [23/3000], Loss: 0.04378453064612889
Epoch [24/3000], L

##### Dados de teste

In [ ]:
massFlowrateTeste = []
PlenumPressureTeste = []
RNN_test = []
x_test = []
alpha_valuesTeste = []
aux1 = []
aux2 = []
alphasTeste = np.random.uniform(0.35,0.65, nAlphasTeste) # Abertura da válvula
N_RotSTeste = np.random.uniform(27e3, 5e4, nAlphasTeste+1)

sim = Simulation(A1, Lc, kv, P1, P_out, C, alphasTeste,N_RotSTeste, nAlphasTeste, nDataTeste, perturb, tempoTeste, dt, interpolation)

sim.run()

RNN_test = sim.RNN_train
massFlowrateTeste = sim.massFlowrate
PlenumPressureTeste = sim.PlenumPressure
alpha_valuesTeste = sim.alpha_values
print(sim.time)

RNN_test = np.array(RNN_test)

for i in range(len(RNN_test) - 3):
    x_test.append(RNN_test[i:i + 3])

x_test = torch.tensor(x_test, dtype=torch.float32)

massFlowrateTeste = np.array(massFlowrateTeste)
PlenumPressureTeste = np.array(PlenumPressureTeste)
interval3 = np.linspace(0, tempoTeste*nAlphasTeste, len(x_test))

[9.11701629 7.05106994]
0.3073873519897461


##### Gráfico Rede Neural

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

tm1 = time.time()
# Colocando o modelo em modo de avaliação

# Supondo que x_test já esteja definido e seja um tensor PyTorch
with torch.no_grad():  
    prediction = model(x_test)

mass = prediction[:, :, 0]
pressure = prediction[:, :, 1]

mass = mass.detach().numpy()
pressure = pressure.detach().numpy()

tm2 = time.time()
print(tm2-tm1)

fig.add_trace(go.Scatter(x=interval3,y=np.squeeze(mass),mode='lines',
                                line=dict(dash='solid')),row=1, col=1)
fig.add_trace(go.Scatter(x=interval3,y=np.squeeze(pressure),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)


for i in range(nAlphasTeste):
    # Modelo
    fig.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(massFlowrateTeste[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(PlenumPressureTeste[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=2)

fig.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig.show()


0.003424406051635742


In [ ]:
massFlowrate100, PlenumPressure100,time2 = model.test_model(x_test, interval3, model)
print(time2)

torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size([1, 3, 4])
torch.Size

In [ ]:
fig3 = make_subplots(rows=1, cols=2, subplot_titles=("Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

fig3.add_trace(go.Scatter(x=interval3,y=np.squeeze(massFlowrate100),mode='lines',
                                line=dict(dash='solid')),row=1, col=1)
fig3.add_trace(go.Scatter(x=interval3,y=np.squeeze(PlenumPressure100),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)

for i in range(nAlphasTeste):
    # Modelo
    fig3.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(massFlowrateTeste[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=1)
    fig3.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(PlenumPressureTeste[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=2)

fig3.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig3.show()